In [15]:
import json
import os
import argparse
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
from hyperparams import STRATEGY
from PLP.DSL import *
from strategy_demonstrations import make_modified_env
from decision_tree_imitation_learning import solve_mouselab
from scipy import stats
from datetime import datetime

import read_csv_to_dataframe as ecf
import experiment_participant_analysis as epa
from importlib import reload

pd.set_option('display.max_rows', 181)
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_colwidth', 60)

# Import data (dataclip)

In [23]:
# import data as csv
exp_version = '1'
dataclip = pd.read_csv('../data/dataclip.csv')


In [24]:
#set up dataframe
reload(ecf)
dataframe1 = ecf.makeDataframe(dataclip, ecf.getParticipantInfoExp1)
dataframe1 = dataframe1.sort_values(by=['condition'], ascending=True)
dataframe1.reset_index(inplace = True, drop=True)

Aborted Experiment: 0 testing trials
Aborted Experiment: 0 testing trials


# Anonymize and store (dataframe_raw)

In [26]:
data = dataframe1.drop(columns=['WorkerId', 'hitId', 'assignmentId','datastring'])
data.apply(lambda x: (json.dumps(a) for a in x) if x.name in 'testingTrials' else x).to_csv('../data/dataframe_raw.csv', index_label='pid')



# Calculate Click Agreement

In [27]:
# define logic formula of strategy
exp_version = '1'
plp_tree = 'lambda st, act : ((among(st, act, lambda st, act:  not(is_leaf(st, act)) ,  lambda st, act, lst: has_best_path(st, act, lst) ) and among(st, act, lambda st, act:  not(is_max_in_branch(st, act))  and  not(is_observed(st, act)) ))) or ((not (among(st, act, lambda st, act:  not(is_leaf(st, act)) ,  lambda st, act, lst: has_best_path(st, act, lst) )) and among(st, act, lambda st, act:  is_max_in_branch(st, act)  and  not(is_observed(st, act)) )))'

# read raw file
dataframe2 = pd.read_csv('../data/dataframe_raw.csv')
dataframe2 = dataframe2.apply(lambda x: (json.loads(a) for a in x) if x.name in 'testingTrials' else x)

if(False):
    # compute
    pas = epa.get_agreement_sample(dataframe2.testingTrials, plp_tree)
    dataframe2['pa_mean'] = pas['click_agreement_means_sample']
    dataframe2['pa_complete'] = pas['click_agreement_ratios_sample']
    dataframe2['pa_et'] = pas['mean_run_lengths_sample']
else:
    # from memory
    pas = pd.read_csv('../data/dataframe_complete.csv')
    dataframe2['pa_mean'] = pas['pa_mean']
    dataframe2['pa_complete'] = pas['pa_complete']
    dataframe2['pa_et'] = pas['pa_et']

# Anonymize and store (dataframe_complete)

In [28]:
data = dataframe2.drop(columns=['testingTrials'])
data.to_csv('../data/dataframe_complete.csv', index=False)